In [1]:
import os
os.chdir('/home/langcongliang/janusq')
from qiskit import QuantumCircuit
from qiskit.quantum_info import random_unitary
from data_objects.circuit import qiskit_to_circuit
from data_objects.backend import  LinearBackend

import time

n_qubits = 5
backend = LinearBackend(n_qubits, 1)

generate a random unitary

In [10]:
unitary = random_unitary(2**n_qubits).data
unitary.shape

(32, 32)

use benchmarking circuit unitary

In [2]:
from data_objects.algorithms import get_algs
unitary = get_algs(n_qubits, backend, ['ghz'], unitary= True)[0]
unitary.shape

(32, 32)

In [3]:
unitary.data

qsd method

In [4]:
from qiskit.synthesis.unitary.qsd import qs_decomposition

start_time =time.time()
qc = qs_decomposition(unitary)

qsd_circuit = qiskit_to_circuit(qc)
qsd_time = time.time() - start_time

QuCT method

In [5]:

from analysis.unitary_decompostion import decompose

start_time = time.time()
quct_circuit = decompose(unitary, allowed_dist = 0.1, backend = backend)
quct_time = time.time() - start_time

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
/home/langcongliang/enter/envs/janusq/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


n_blocks of 5 qubits should be 4
n_blocks of 5 qubits should be 4


2024-03-26 07:40:48,981	INFO worker.py:1724 -- Started a local Ray instance.
(pid=1258138) An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


(find_parmas_remote pid=1258138) Epoch:     0 | Loss: 0.00001  | Dist: 0.00001 | Time: 0.715
(find_parmas_remote pid=1258138) Epoch:   100 | Loss: 0.00003  | Dist: 0.00003 | Time: 0.004
(find_parmas_remote pid=1258138) Epoch:   200 | Loss: 0.00001  | Dist: 0.00001 | Time: 0.004
(find_parmas_remote pid=1258138) Epoch:   300 | Loss: 0.00000  | Dist: 0.00000 | Time: 0.004
(find_parmas_remote pid=1258138) Epoch:   400 | Loss: 0.00002  | Dist: 0.00002 | Time: 0.004
(find_parmas_remote pid=1258138) Epoch:   500 | Loss: 0.00000  | Dist: 0.00000 | Time: 0.004
(find_parmas_remote pid=1258138) Epoch:   600 | Loss: 0.00000  | Dist: 0.00000 | Time: 0.009
(find_parmas_remote pid=1258138) Epoch:   700 | Loss: 0.00001  | Dist: 0.00001 | Time: 0.009
(find_parmas_remote pid=1258138) Epoch:   800 | Loss: 0.00000  | Dist: 0.00000 | Time: 0.009
(find_parmas_remote pid=1258138) Epoch:   900 | Loss: 0.00000  | Dist: 0.00000 | Time: 0.009
(find_parmas_remote pid=1258138) Epoch:  1000 | Loss: 0.00000  | Dist:

  0%|          | 0/1 [00:00<?, ?it/s](pid=1257973) An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
(pid=1258024) An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
(decompose_to_gates_remote pid=1258138) iter_count= 0 now_dist= 0.2426698272970188, #gate = 11 
(decompose_to_gates_remote pid=1258138) 
(pid=1257313) An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu. [repeated 133x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(decompose_to_gates_remote pid=1258138) iter_count= 1 now_dist= 0.20241892716023868, #gate = 34  [repeated 5x across cluster]
(decompose_to_gates_remote pid=1258138)  [repeated 5x across cluster]
(decompose_to_gates_remote pi

qfast method

In [6]:
from bqskit.compiler import Compiler
from bqskit.ir.circuit import Circuit
from bqskit.ir.gates import CircuitGate
from bqskit.ir.gates.constant.cx import CNOTGate
from bqskit.ir.gates.parameterized.u3 import U3Gate
from bqskit.passes import QFASTDecompositionPass
from bqskit.qis import UnitaryMatrix
from bqskit.ext import bqskit_to_qiskit        
circuit = Circuit(2)
circuit.append_gate(CNOTGate(), (0, 1))
circuit.append_gate(U3Gate(), (0,))
cg = CircuitGate(circuit)
circuit = Circuit.from_unitary(unitary)

start_time = time.time()
qfast = QFASTDecompositionPass(cg)
circuit.perform(qfast)
dist = circuit.get_unitary().get_distance_from(unitary)
assert dist <= 1e-5

qc = bqskit_to_qiskit(circuit)
qfast_circuit = qiskit_to_circuit(qc)
qfast_time = time.time() - start_time

In [ ]:

synthesis_method_result = [qsd_circuit, qfast_circuit, quct_circuit]
synthesis_method_time = [qsd_time, qfast_time, quct_time]
for res, tim in synthesis_method_result, synthesis_method_time:
    print(f"#gate: {res.n_gates}, depth: {res.depth}, time: {tim} \n")